In [1]:
import mplsoccer.statsbomb as sbapi
import pandas as pd
import os
import glob

This notebook takes the StatsBomb json files and turns them into parquet files. These are extremely fast to load so good for this prototyping kind of analysis.

# Change these paths/ parameters
You will need to change these paths/ parameters depending on where the StatsBomb open-data is located, how and where you want to save the resulting data, and if you only want the new files to be processed.

In [2]:
# open data folder is one folder down in the directory. To change if run elsewhere
STATSBOMB_DATA = os.path.join('..', '..', '..', 'open-data','data')
# save files in folder in current directory. To change if want to save elsewhere
DATA_FOLDER = os.path.join('..', '..', 'data', 'statsbomb')

# Get the data file paths

In [3]:
event_links = glob.glob(os.path.join(STATSBOMB_DATA, 'events', '**', '*.json'),recursive=True)
lineup_links = glob.glob(os.path.join(STATSBOMB_DATA, 'lineups', '**', '*.json'),recursive=True)
match_links = glob.glob(os.path.join(STATSBOMB_DATA, 'matches', '**', '*.json'),recursive=True)
competition_path = os.path.join(STATSBOMB_DATA, 'competitions.json')

# Make the directory structure

In [4]:
# make the directory structure
for folder in ['event_raw', 'related_event_raw', 'freeze_frame_raw', 'tactic_raw', 'lineup_raw']:
    path = os.path.join(DATA_FOLDER, folder)
    if not os.path.exists(path):
        os.mkdir(path)

# Read the competition datam

In [5]:
df_competition = sbapi.read_competition(competition_path, warn=False)
# note there is a slight loss of data quality with timestamps, but these aren't relevant for analysis
# pandas has nanoseconds, which aren't supported in parquet (supports milliseconds)
df_competition.to_parquet(os.path.join(DATA_FOLDER, 'competition.parquet'), allow_truncated_timestamps=True)
df_competition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   competition_id      36 non-null     int64         
 1   season_id           36 non-null     int64         
 2   country_name        36 non-null     object        
 3   competition_name    36 non-null     object        
 4   competition_gender  36 non-null     object        
 5   season_name         36 non-null     object        
 6   match_updated       36 non-null     datetime64[ns]
 7   match_available     36 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 2.4+ KB


# Read the match data

In [6]:
match_dfs = [sbapi.read_match(file, warn=False) for file in match_links]
df_match = pd.concat(match_dfs)
# again there is a slight loss of quality when saving timestamps, but only relevant for last_updated
df_match.to_parquet(os.path.join(DATA_FOLDER, 'match.parquet'), allow_truncated_timestamps=True)
df_match.info()

Skipping ..\..\..\open-data\data\matches\16\4.json: empty json
Skipping ..\..\..\open-data\data\matches\16\42.json: empty json
Skipping ..\..\..\open-data\data\matches\16\44.json: empty json
Skipping ..\..\..\open-data\data\matches\16\76.json: empty json
<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 0 to 51
Data columns (total 48 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   match_id                         842 non-null    int64         
 1   match_date                       842 non-null    datetime64[ns]
 2   kick_off                         841 non-null    datetime64[ns]
 3   home_score                       842 non-null    int64         
 4   away_score                       842 non-null    int64         
 5   last_updated                     842 non-null    datetime64[ns]
 6   match_week                       842 non-null    int64         
 7   competition_id 

# Read the lineup data

In [7]:
LINEUP_FOLDER = os.path.join(DATA_FOLDER, 'lineup_raw')
# loop through the links and store as parquet files - small and fast files
for file in lineup_links:
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    if not os.path.isfile(os.path.join(LINEUP_FOLDER, save_path)):
        try:
            print('Trying:', file)
            df_lineup = sbapi.read_lineup(file, warn=False)
            df_lineup.to_parquet(os.path.join(LINEUP_FOLDER, save_path))
        except:
            print('Skipping:', file)
            pass

Convert to a single dataframe

In [8]:
lineup_files = glob.glob(os.path.join(LINEUP_FOLDER, '*.parquet'))
df_lineup = pd.concat([pd.read_parquet(file) for file in lineup_files])
# replace some ids that appear to be duplicated. Then de-duplicate
df_lineup.player_id.replace({18103: 38522,  # Dietmar Hamann
                             17275: 4656,  # Hannah Jayne Blundell
                             17524: 4655,  # Jennifer Beattie
                             10172: 4644,  # Jill Scott
                             4634: 5088,  # Crystal Dunn
                             }, inplace=True)
df_lineup.to_parquet(os.path.join(DATA_FOLDER, 'lineup.parquet'))
df_lineup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25004 entries, 0 to 35
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   team_id               25004 non-null  int64  
 1   team_name             25004 non-null  object 
 2   match_id              25004 non-null  int64  
 3   player_id             25004 non-null  int64  
 4   player_name           25004 non-null  object 
 5   player_nickname       13399 non-null  object 
 6   player_jersey_number  25004 non-null  int64  
 7   player_country_id     24999 non-null  float64
 8   player_country_name   24999 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 1.9+ MB


# Read the event data

In [9]:
# loop through the links and store as parquet files - small and fast files
for file in event_links:
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    if not os.path.isfile(os.path.join(DATA_FOLDER, 'event_raw', save_path)):
        try:
            print('Trying:', file)
            dict_event = sbapi.read_event(file, warn=False)
            # save to parquet files
            # using the dictionary key to access the dataframes from the dictionary
            dict_event['event'].to_parquet(os.path.join(DATA_FOLDER, 'event_raw', save_path))
            dict_event['related_event'].to_parquet(os.path.join(DATA_FOLDER, 'related_event_raw', save_path))
            dict_event['shot_freeze_frame'].to_parquet(os.path.join(DATA_FOLDER, 'freeze_frame_raw', save_path))
            dict_event['tactics_lineup'].to_parquet(os.path.join(DATA_FOLDER, 'tactic_raw', save_path))
        except:
            print('Skipping:', file)
            pass

Single dataframe events

In [10]:
event_files = glob.glob(os.path.join(DATA_FOLDER, 'event_raw', '*.parquet'))
df_event = pd.concat([pd.read_parquet(file) for file in event_files])
df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
df_event.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3056120 entries, 0 to 3746
Data columns (total 130 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   match_id                          3056120 non-null  int64  
 1   id                                3056120 non-null  object 
 2   index                             3056120 non-null  int64  
 3   period                            3056120 non-null  int64  
 4   timestamp_minute                  3056120 non-null  int64  
 5   timestamp_second                  3056120 non-null  int64  
 6   timestamp_millisecond             3056120 non-null  int64  
 7   minute                            3056120 non-null  int64  
 8   second                            3056120 non-null  int64  
 9   type_id                           3056120 non-null  int64  
 10  type_name                         3056120 non-null  object 
 11  outcome_id                        42817

Single dataframe shot freeze frames

In [11]:
freeze_files = glob.glob(os.path.join(DATA_FOLDER, 'freeze_frame_raw', '*.parquet'))
df_freeze = pd.concat([pd.read_parquet(file) for file in freeze_files])
df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
df_freeze.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266478 entries, 0 to 341
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    266478 non-null  object 
 1   event_freeze_id       266478 non-null  int64  
 2   player_teammate       266478 non-null  bool   
 3   player_id             266478 non-null  int64  
 4   player_name           266478 non-null  object 
 5   player_position_id    266478 non-null  int64  
 6   player_position_name  266478 non-null  object 
 7   x                     266478 non-null  float64
 8   y                     266478 non-null  float64
 9   match_id              266478 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 20.6+ MB


Single dataframe tactics

In [12]:
tactic_files = glob.glob(os.path.join(DATA_FOLDER, 'tactic_raw', '*.parquet'))
df_tactic = pd.concat([pd.read_parquet(file) for file in tactic_files])
df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
df_tactic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35225 entries, 0 to 32
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    35225 non-null  object 
 1   event_tactics_id      35225 non-null  int64  
 2   player_jersey_number  35221 non-null  float64
 3   player_id             35225 non-null  int64  
 4   player_name           35225 non-null  object 
 5   player_position_id    35225 non-null  int64  
 6   player_position_name  35225 non-null  object 
 7   match_id              35225 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 2.4+ MB


Single dataframe related events

In [13]:
related_files = glob.glob(os.path.join(DATA_FOLDER, 'related_event_raw', '*.parquet'))
df_related = pd.concat([pd.read_parquet(file) for file in related_files])
df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
df_related.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5941216 entries, 0 to 7467
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   id                 5941216 non-null  object
 1   id_related         5941216 non-null  object
 2   type_name          5941216 non-null  object
 3   index              5941216 non-null  int64 
 4   type_name_related  5941216 non-null  object
 5   index_related      5941216 non-null  int64 
 6   match_id           5941216 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 362.6+ MB
